### 📊 Redfin Housing Market Analysis Workflow

#### [1. 기획] -> [2. 수집] -> [3. 정제] -> [4. 변수 선택]

---

### Phase 1: 기획 및 데이터 수집 (Planning & Acquisition)
#### 1. 분석 대상 선정 (Region Selection)

**대상:** 워싱턴주 시애틀 광역권의 킹 카운티(King County) vs. 피어스 카운티(Pierce County)

**선정 이유**
- 경제적 연결성: 시애틀(King)은 일자리 중심지, 타코마(Pierce)는 위성 도시
- 비교 용이성: "도심(Expensive) vs. 외곽(Affordable)" 구조가 명확

#### 2. 데이터 수집 (Data Collection)

**소스:** Redfin "Sold" Data (최근 6개월)

---

### Phase 2: 데이터 전처리 (Preprocessing & Filtering)
#### 3. 데이터 병합 및 라벨링 (Merging)

**작업:** 두 CSV 파일 병합 + COUNTY 컬럼 생성

#### 4. 주택 유형 필터링 (Property Type Filtering)

**유지:** Single Family Residential, Condo/Co-op, Townhouse

**제거:** Mobile/Manufactured Home, Multi-Family

#### 5. 상식 필터링 (Common Sense Filtering)

제거 조건: PRICE < $50k, BEDS < 1, SQUARE FEET < 300


In [3]:
# 1. 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ---------------------------------------------------------
# Step 1: 데이터 로드 및 병합 (Phase 1)
# ---------------------------------------------------------
# (코랩에 파일 업로드 후 경로를 맞춰주세요)
king = pd.read_csv('/content/drive/MyDrive/Data_Science/Redfin/King_County.csv') # 파일명 변경 필요
pierce = pd.read_csv('/content/drive/MyDrive/Data_Science/Redfin/Pierce_County.csv') # 파일명 변경 필요

# 구분자 컬럼 추가
king['COUNTY'] = 'King'
pierce['COUNTY'] = 'Pierce'

# 병합
df = pd.concat([king, pierce], ignore_index=True)
print(f"최초 데이터 개수: {len(df)}개")

# ---------------------------------------------------------
# Step 2: 행(Row) 필터링 - 데이터 정제 (Phase 2)
# ---------------------------------------------------------

# 1. 주택 유형 필터링 (유효한 3가지 유형만 유지)
valid_types = ['Single Family Residential', 'Condo/Co-op', 'Townhouse']
df_clean = df[df['PROPERTY TYPE'].isin(valid_types)].copy()
print(f"주택 유형 필터링 후: {len(df_clean)}개 (Mobile/Multi-family 제거)")

# 2. 상식 필터링 (오류 데이터 제거)
# 조건: 면적 300 이상, 가격 5만불 이상, 침실 1개 이상
df_clean = df_clean[
    (df_clean['SQUARE FEET'] > 300) &
    (df_clean['PRICE'] > 50000) &
    (df_clean['BEDS'] >= 1)
].copy()
print(f"상식 필터링 후: {len(df_clean)}개 (오류 데이터 제거)")

# 3. 이상치 제거 (가격 상위 1% 제거)
price_cutoff = df_clean['PRICE'].quantile(0.99)
df_final = df_clean[df_clean['PRICE'] <= price_cutoff].copy()
print(f"상위 1% 제거 후: {len(df_final)}개 (최종 분석 데이터)")

# ---------------------------------------------------------
# Step 3: 열(Column) 변수 가공 (Phase 3)
# ---------------------------------------------------------

# 1. 변수 제거 (신뢰할 수 없거나 빈 변수)
# LOT SIZE (콘도 결측치), DAYS ON MARKET (100% 결측치) 등을 제외하고
# 분석에 필요한 핵심 컬럼만 선택
selected_columns = [
    'PRICE', 'SQUARE FEET', 'YEAR BUILT',
    'BEDS', 'BATHS', 'PROPERTY TYPE', 'COUNTY'
]
df_final = df_final[selected_columns].copy()

# 2. 파생 변수 생성
# AGE (건물 연식)
df_final['AGE'] = 2025 - df_final['YEAR BUILT']

# log_PRICE (로그 변환 가격 - 회귀분석용)
df_final['log_PRICE'] = np.log(df_final['PRICE'])

# Price_per_SqFt (평당 가격 - 비교 분석용)
df_final['Price_per_SqFt'] = df_final['PRICE'] / df_final['SQUARE FEET']

# ---------------------------------------------------------
# 최종 확인
# ---------------------------------------------------------
print("\n--- 최종 데이터셋 요약 ---")
print(df_final.info())
display(df_final.head())

최초 데이터 개수: 702개
주택 유형 필터링 후: 670개 (Mobile/Multi-family 제거)
상식 필터링 후: 667개 (오류 데이터 제거)
상위 1% 제거 후: 660개 (최종 분석 데이터)

--- 최종 데이터셋 요약 ---
<class 'pandas.core.frame.DataFrame'>
Index: 660 entries, 1 to 701
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PRICE           660 non-null    float64
 1   SQUARE FEET     660 non-null    float64
 2   YEAR BUILT      660 non-null    float64
 3   BEDS            660 non-null    float64
 4   BATHS           660 non-null    float64
 5   PROPERTY TYPE   660 non-null    object 
 6   COUNTY          660 non-null    object 
 7   AGE             660 non-null    float64
 8   log_PRICE       660 non-null    float64
 9   Price_per_SqFt  660 non-null    float64
dtypes: float64(8), object(2)
memory usage: 56.7+ KB
None


,PRICE,SQUARE FEET,YEAR BUILT,BEDS,BATHS,PROPERTY TYPE,COUNTY,AGE,log_PRICE,Price_per_SqFt
1,407000.0,923.0,1980.0,2.0,1.0,Condo/Co-op,King,45.0,12.916568,440.953413
2,1300000.0,1065.0,1963.0,2.0,1.0,Condo/Co-op,King,62.0,14.077875,1220.657277
3,825000.0,1690.0,1941.0,3.0,2.0,Single Family Residential,King,84.0,13.623139,488.165680
4,819900.0,1685.0,2025.0,4.0,2.5,Townhouse,King,0.0,13.616938,486.587537
6,470000.0,1670.0,1963.0,6.0,2.0,Single Family Residential,King,62.0,13.060488,281.437126
